In [1]:
from requests import get, Session, request
from requests.compat import urlparse, urlunparse, urljoin
from bs4 import BeautifulSoup
import re
headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}

In [3]:
def robotParser(domain):
    url = urlunparse(urlparse(domain)[:2] + ('',)*4)
    url += '/robots.txt'    
    pathEnable = dict()
    resp = get(url)
    if resp.status_code == 200:
        agent = None
        for line in resp.text.splitlines():
            k, *v = line.split(':')
            k = k.strip()
            v = ':'.join(v).strip()
            if k.lower() == 'user-agent':
                agent = v
                if v not in pathEnable:
                    pathEnable[v] = dict()
            else:
                if k.lower() == 'allow':
                    pathEnable[agent][v] = True
                else:
                    pathEnable[agent][v] = False
    else:
        pathEnable['*'] = True
    return pathEnable

In [4]:
def canFetch(pathEnable, path):
    agent = '*'
    path = urlparse(path).path
    
    if agent in pathEnable:
        if path in pathEnable[agent]:
            return pathEnable[agent][path]
        else:
            if path == '/':
                return True
            else:
                return canFetch(pathEnable,
                                '/'.join(path.split('/')[:-1]))
    else:
        return True

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
url = 'https://www.yna.co.kr/'
urls = list()
urls.append((url, 0)) # 깊이 제한
seens = list()
headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}

robots = dict()

categories = ['정치', '경제', '사회', '금융증권산업', '사건사고', '문화', '생활건강', 'IT과학', '북한', '국제', '스포츠', '연예']

while urls:
    # URLs Pool
    seed = urls.pop(-1) # Queue=BFS, Stack=DFS
    seens.append(seed[0]) # 깊이 제한
    
    # Robots.txt
#     if seed not in robots:
#         robots[seed] = robotParser(seed)
#     rp = robots[seed]
#     print('[Robots.txt]', seed, canFetch(rp, seed)) # True일때만,지금은무시
    
    # Focused Crawling
#     if seed[1] > 2: # 자의적 => 휴리스틱
#         continue
        
    resp = get(seed[0], headers=headers) # 깊이 제한
    if resp.status_code == 200 and 'Content-Type'.lower() in resp.headers:
                                   # content-type이 있을때만
        if 'text/html' in\
           [_.strip() for _ in resp.headers['Content-Type'].split(';')]: #바꿔야함
            dom = BeautifulSoup(resp.text, 'html.parser')
            aList = dom.select('a[role=menuitem]')
            hList = dom.select('a.cluster_text_headline')
            news = dom.select_one('#ct #contents')
            if len(aList) > 0:
                for el in aList[1:6]:
                    url = el.attrs['href']
                    nextUrl = urljoin(seed[0], url)
                    urlParams = urlparse(nextUrl)

                    # URL 체크
                    if nextUrl not in seens and\
                       nextUrl not in [_[0] for _ in urls] and\
                       nextUrl.startswith('http'):
                        urls.append((nextUrl, seed[1]+1))
            if len(hList) > 0:
                for el in hList:
                    url = el.attrs['href']
                    nextUrl = urljoin(seed[0], url)
                    urlParams = urlparse(nextUrl)
# https://n.news.naver.com/mnews/article/119/0002640090?sid=102
                    # URL 체크
                    if nextUrl not in seens and\
                       nextUrl not in [_[0] for _ in urls] and\
                       nextUrl.startswith('http'):
                        urls.append((nextUrl, seed[1]+1))
            news = dom.select_one('#ct #contents')
            if news:
                fileName = re.search('(\d{10})[?]sid=(\d{3})$', resp.url)
                with open(
                    '/content/drive/MyDrive/dataset/paper/{}-{}.txt'.format(fileName.group(2),
                                              fileName.group(1)),
                    'w', encoding='utf8') as f:
                    f.write(news.text.strip())
                for img in news.select('img[data-src]'):
                    urls.append((urljoin(resp.url, img.attrs['data-src']), seed[1]+1))
#             for el in dom.select('[src], [href]'): # [:10] 너비제한
#                 url = el.attrs['src' if 'src' in el.attrs else 'href']
#                 nextUrl = urljoin(seed[0], url) # 깊이 제한
#                 urlParams = urlparse(nextUrl)

#                 # URL 체크
#                 if nextUrl not in seens and\
#                    nextUrl not in [_[0] for _ in urls] and\
#                    urlParams.netloc == 'blog.naver.com' and\
#                    nextUrl.startswith('http'):
#                     # javascript, #fragment 제외시켜야 함
#                     # Focused = 특정 도메인으로 제한
#                     urls.append((nextUrl, seed[1]+1)) # 깊이 제한
        else:
            #image/jpeg
            ext = re.search(r'image/(png|jpeg|bmp|gif)', resp.headers['Content-Type'])
            if ext:
#                 fileName = re.search('(\d{8}_\d{3})', resp.url)
                fileName = resp.url.split('/')[-1]
                fileName += '.'+ ext.group(1)
                with open('/content/drive/MyDrive/dataset/news/'+fileName, 'wb') as fp:
                    fp.write(resp.content)
    print(len(urls), len(seens))

5 1
28 2
31 3
30 4
29 5
28 6
27 7
32 8
31 9
30 10
29 11
28 12
27 13
26 14
33 15
32 16
31 17
30 18
29 19
28 20
27 21
26 22
25 23
30 24
29 25
28 26
27 27
26 28
25 29
24 30
26 31
25 32
24 33
23 34
26 35
25 36
24 37
23 38
22 39
24 40
23 41
22 42
21 43
23 44
22 45
21 46
20 47
22 48
21 49
20 50
19 51
20 52
19 53
18 54
20 55
19 56
18 57
17 58
20 59
19 60
18 61
17 62
16 63
19 64
18 65
17 66
16 67
15 68
16 69
15 70
14 71
14 72
13 73
14 74
13 75
12 76
14 77
13 78
12 79
11 80
14 81
13 82
12 83
11 84
10 85
14 86
13 87
12 88
11 89
10 90
9 91
12 92
11 93
10 94
9 95
8 96
10 97
9 98
8 99
7 100
9 101
8 102
7 103
6 104
7 105
6 106
5 107
6 108
5 109
4 110
29 111
33 112
32 113
31 114
30 115
29 116
28 117
33 118
32 119
31 120
30 121
29 122
28 123
27 124
32 125
31 126
30 127
29 128
28 129
27 130
26 131
31 132
30 133
29 134
28 135
27 136
26 137
25 138
29 139
28 140
27 141
26 142
25 143
24 144
31 145
30 146
29 147
28 148
27 149
26 150
25 151
24 152
23 153
27 154
26 155
25 156
24 157
23 158
22 159
28 160
27 16